In [55]:
import json
import logging
import pdb

import requests

from typing import List
from pydantic import BaseModel


In [2]:
logger = logging.getLogger('dev')
logger.setLevel(logging.INFO)

In [3]:
GRANT_TYPES = {
    "authorization_code": "authorization_code",
    "refresh_token": "refresh_token"
}
zoho_self_client_dict = {
    "code": "1000.4aa0e249d8afbc3a712bb4d25730d143.1499b2a8b23e0fb094d3e1e8d018689d",
    "scope": ["Desk.tickets.ALL"],
    "expiry_time": 1610367630253,
    "client_id": "1000.QXWN1SUWKQC7F1F1D4C2R8BC00I3AE",
    "client_secret": "6a27c69dfb8e31e6a9ceea8ef5bf152fd41aea8c07",
    "grant_types": GRANT_TYPES,
    "redirect_uri": "https://patient.serenity.health",
}
zoho_urls = {
    "oauth": "https://accounts.zoho.com/oauth/v2/token",
    "base_url": "https://desk.zoho.com/api/v1",
}

# state=-5466400890088961855&code=1000.71f7dd93b5571315574f9b82315ea6d6.91844bdb4a86740d3dba8a7e5b655d40&location=us

In [4]:
class ZohoSelfClient(BaseModel):
    scope: List[str]
    expiry_time: int = None
    client_id: str
    client_secret: str
    code: str
    grant_types: dict
    access_token: str = None
    refresh_token: str = None
    redirect_uri: str = "https://patient.serenity.health"
        
    def get_authorization_header(self) -> str:
        return f"Zoho-oauthtoken {self.access_token}"


In [5]:
zoho_self_client = ZohoSelfClient(**zoho_self_client_dict)
# zoho_self_client.code = "1000.edcce9870215d045cdfd6a6dd475b5c6.7adfb9082839d51d808228615cc3babb"

In [6]:
def get_zoho_oauth_tokens(zoho_self_client: ZohoSelfClient, oauth_url: str = zoho_urls["oauth"])-> dict: 
    payload = {
        "code": zoho_self_client.code,
        "grant_type": zoho_self_client.grant_types["authorization_code"],
        "client_id": zoho_self_client.client_id,
        "client_secret": zoho_self_client.client_secret,
        "redirect_uri": zoho_self_client.redirect_uri,
    }
    response = requests.post(oauth_url, params=payload)
    try:
        if response.status_code == 200 and not response.json().get("error"):
            logging.info("Successfully retrieved oauth tokens")
            tokens = response.json()
            zoho_self_client.access_token = tokens["access_token"]
            zoho_self_client.refresh_token = tokens["refresh_token"]
            tokens["status"] = True
        else:
            logging.error("Failed to retrieve oauth tokens")
            tokens = dict()
            tokens["status"] = False
    except (ValueError, KeyError) as e:
        pass # there's no token
    return tokens

In [7]:
zoho_self_client_dict

{'code': '1000.4aa0e249d8afbc3a712bb4d25730d143.1499b2a8b23e0fb094d3e1e8d018689d',
 'scope': ['Desk.tickets.ALL'],
 'expiry_time': 1610367630253,
 'client_id': '1000.QXWN1SUWKQC7F1F1D4C2R8BC00I3AE',
 'client_secret': '6a27c69dfb8e31e6a9ceea8ef5bf152fd41aea8c07',
 'grant_types': {'authorization_code': 'authorization_code',
  'refresh_token': 'refresh_token'},
 'redirect_uri': 'https://patient.serenity.health'}

In [12]:
tokens = get_zoho_oauth_tokens(zoho_self_client)
zoho_self_client.dict()

{'scope': ['Desk.tickets.ALL'],
 'expiry_time': 1610367630253,
 'client_id': '1000.QXWN1SUWKQC7F1F1D4C2R8BC00I3AE',
 'client_secret': '6a27c69dfb8e31e6a9ceea8ef5bf152fd41aea8c07',
 'code': '1000.4aa0e249d8afbc3a712bb4d25730d143.1499b2a8b23e0fb094d3e1e8d018689d',
 'grant_types': {'authorization_code': 'authorization_code',
  'refresh_token': 'refresh_token'},
 'access_token': '1000.047275292bd89ec13144ad7454eb201b.26600ff92420957e56ff23c0d7a765de',
 'refresh_token': '1000.87583d1a480fb5a74088ca3d9f5d536f.6f32f768e0780688e75bf742bd6238be',
 'redirect_uri': 'https://patient.serenity.health'}

In [13]:
zoho_self_client.get_authorization_header()

'Zoho-oauthtoken 1000.047275292bd89ec13144ad7454eb201b.26600ff92420957e56ff23c0d7a765de'

In [14]:
def refresh_zoho_oauth_tokens(zoho_self_client: ZohoSelfClient, oauth_url: str = zoho_urls["oauth"])-> dict: 
    if not self_client.refresh_token:
        tokens = get_zoho_oauth_tokens(zoho_self_client)
        if not tokens["status"]:
            return tokens
    else:
        payload = {
            "refresh_token": zoho_self_client.refresh_token,
            "scope": zoho_self_client.scope,
            "grant_type": zoho_self_client.grant_types["refresh_token"],
            "client_id": zoho_self_client.client_id,
            "client_secret": zoho_self_client.client_secret,
            "redirect_uri": zoho_self_client.redirect_uri,
        }
        response = requests.post(oauth_url, params=payload)
        logging.warning(response.url)
        try:
            if response.status_code == 200 and not response.json().get("error"):
                tokens = response.json()
                zoho_self_client.access_token = tokens["access_token"]
                tokens["status"] = True
            else:
                tokens = dict()
                tokens["status"] = False
        except ValueError:
            pass # there's no token
        return tokens

# serenity_zoho_access_keys

In [ ]:
tokens = refresh_zoho_oauth_tokens(zoho_self_client)
zoho_self_client.dict()

## Get departments

In [15]:
zoho_self_client.get_authorization_header()

'Zoho-oauthtoken 1000.047275292bd89ec13144ad7454eb201b.26600ff92420957e56ff23c0d7a765de'

In [28]:
def get_departments(
        zoho_self_client: ZohoSelfClient, 
        params: dict = {}, 
        base_url: str = zoho_urls["base_url"],
    ) -> dict:
    headers = {"Authorization": f"Zoho-oauthtoken {zoho_self_client.access_token}"}
#     params = {**params, 'isEnabled': True, "chatStatus": "AVAILABLE"}
    response = requests.get(f"{base_url}/departments", headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
    else:
        data = {"error": True}
    return data

In [35]:
def get_department(
        zoho_self_client: ZohoSelfClient, 
        department_id: int,
        params: dict = {}, 
        base_url: str = zoho_urls["base_url"],
    ) -> dict:
    headers = {"Authorization": f"Zoho-oauthtoken {zoho_self_client.access_token}"}
#     params = {**params, 'isEnabled': True, "chatStatus": "AVAILABLE"}
    response = requests.get(f"{base_url}/departments/{department_id}", headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
    else:
        data = {"error": True}
    return data

In [36]:
departments = get_department(zoho_self_client, 286477000000006907)
departments

{'id': '286477000000006907',
 'name': 'clearspacelabs',
 'description': None,
 'createdTime': '2018-05-31T23:20:04.000Z',
 'associatedAgentIds': ['286477000001037001', '286477000000086033'],
 'nameInCustomerPortal': 'clearspacelabs',
 'hasLogo': False,
 'creatorId': '286477000000086033',
 'chatStatus': 'NOT_CREATED',
 'sanitizedName': 'clearspacelabs',
 'isAssignToTeamEnabled': True,
 'isVisibleInCustomerPortal': True,
 'isEnabled': True,
 'isDefault': True}

In [26]:
!curl -X GET "https://desk.zoho.com/api/v1/departments"\
  -H "Authorization:Zoho-oauthtoken 1000.047275292bd89ec13144ad7454eb201b.26600ff92420957e56ff23c0d7a765de"

{"data":[{"id":"286477000000006907","name":"clearspacelabs","description":null,"createdTime":"2018-05-31T23:20:04.000Z","nameInCustomerPortal":"clearspacelabs","hasLogo":false,"creatorId":"286477000000086033","chatStatus":"NOT_CREATED","isVisibleInCustomerPortal":true,"isAssignToTeamEnabled":true,"sanitizedName":"clearspacelabs","isEnabled":true,"isDefault":true}]}

## Get all tickets

In [ ]:
!curl -X GET https://desk.zoho.com/api/v1/tickets?include=contacts,assignee,departments,team,isRead\
  -H "Authorization:Zoho-oauthtoken 1000.62364743ad3344f15fe51a8988eaef42.abd218be429b372f12bad73af6e5b8ac"

In [ ]:
def get_all_tickets(zoho_self_client: ZohoSelfClient, params: dict = {}, base_url: str = zoho_urls["base_url"],) -> dict:    
    headers = {"Authorization": f"Zoho-oauthtoken {zoho_self_client.access_token}"}
    response = requests.get(f"{base_url}/tickets", headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
    else:
        data = {"error": True}
    return data

In [56]:
def create_ticket(zoho_self_client: ZohoSelfClient, payload:dict, base_url: str = zoho_urls["base_url"],) -> dict:    
    headers = {"Authorization": f"Zoho-oauthtoken {zoho_self_client.access_token}"}
    response = requests.post(f"{base_url}/tickets", headers=headers, data=json.dumps(payload))
    if response.status_code == 201:
        data = response.json()
    else:
        data = {"error": True}
    return data, response

In [64]:
response = create_ticket(zoho_self_client, payload = ticket_data)

In [66]:
response[1].json()

{'modifiedTime': '2021-01-12T13:01:12.000Z',
 'subCategory': None,
 'statusType': 'Open',
 'subject': 'Test automated ticket generation',
 'dueDate': None,
 'departmentId': '286477000000006907',
 'channel': 'Phone',
 'onholdTime': None,
 'language': None,
 'source': {'appName': None,
  'extId': None,
  'permalink': None,
  'type': 'SYSTEM',
  'appPhotoURL': None},
 'resolution': None,
 'sharedDepartments': [],
 'closedTime': None,
 'approvalCount': '0',
 'isTrashed': False,
 'createdTime': '2021-01-12T13:01:12.000Z',
 'id': '286477000001053021',
 'isResponseOverdue': False,
 'customerResponseTime': '2021-01-12T13:01:12.000Z',
 'productId': None,
 'contactId': '286477000001053001',
 'threadCount': '0',
 'secondaryContacts': [],
 'priority': None,
 'classification': None,
 'commentCount': '0',
 'taskCount': '0',
 'accountId': None,
 'phone': None,
 'webUrl': 'https://desk.zoho.com/support/serenityhealth/ShowHomePage.do#Cases/dv/286477000001053021',
 'isSpam': False,
 'status': 'Open',
 '

In [63]:
ticket_data = {
    "subject": "Test automated ticket generation",
    "departmentId": 286477000000006907,
    "email": "chris@clearspacelabs.com",
    "contact": {
        "email": "chris@clearspacelabs.com"
    },
    "description" : "Description of automated ticket",
    "language" : "English",
    "category" : "general",
    "status" : "Open",
    "cf":{},
}

In [41]:
null = None

In [52]:
ticket_data = {
    "subCategory" : "Serenity Patient portal",
    "cf" : {
    "cf_permanentaddress" : null,
    "cf_dateofpurchase" : null,
    "cf_phone" : null,
    "cf_numberofitems" : null,
    "cf_url" : null,
    "cf_secondaryemail" : null,
    "cf_severitypercentage" : "0.0",
    "cf_modelname" : "F3 2017"
    },
    "productId" : "",
    "contactId" : "",
    "contact":{
        "email": "chris@clearspacelabs.com"
    },
    "subject" : "Test automated ticket creation",
    "dueDate" : "2021-01-21T16:16:16.000Z",
    "departmentId" : "286477000000006907",
    "channel" : "Email",
    "description" : "Description of automated",
    "language" : "English",
    "priority" : "High",
    "classification" : "",
    "assigneeId" : "",
    "phone" : "+250786745117",
    "category" : "general",
    "email" : "chris@clearspacelabs.com",
    "status" : "Open"
}